In [26]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

from scipy.stats import uniform, randint, loguniform
from sklearn.model_selection import RepeatedStratifiedKFold

In [2]:
sns.set_style("whitegrid", {'grid.linestyle': '--'})
sns.set_style({'font.family':'serif', 'font.serif':'Computer Modern'})
sns.set_context(font_scale=2, rc={"font.size":10,"axes.titlesize":20,"axes.labelsize":15})

In [3]:
df_train = pd.read_csv("df_train_processed.csv")
df_test = pd.read_csv("df_test_processed.csv")

In [4]:
df_train.head()

,vocal_channel,emotion,emotional_intensity,statement,repetition,actor,sex,frame_count,sum,mean,...,stft_std_w4,stft_min_w4,stft_q01_w4,stft_q05_w4,stft_q25_w4,stft_q50_w4,stft_q75_w4,stft_q95_w4,stft_kur_w4,stft_skew_w4
0,speech,neutral,normal,Kids are talking by the door,1st,1,M,158558,0.145081,9.150000e-07,...,0.155455,0.000000,0.000000,0.566462,0.709962,0.799141,0.896606,1.0,6.742190,-1.706215
1,speech,neutral,normal,Kids are talking by the door,2nd,1,M,160160,0.114319,7.137790e-07,...,0.163183,0.270133,0.368623,0.472736,0.623183,0.744908,0.874713,1.0,-0.700420,-0.201495
2,speech,neutral,normal,Dogs are sitting by the door,1st,1,M,156956,0.149963,9.554485e-07,...,0.190634,0.000000,0.000000,0.417919,0.643636,0.774253,0.899156,1.0,1.688986,-1.024773
3,speech,neutral,normal,Dogs are sitting by the door,2nd,1,M,152152,0.139618,9.176213e-07,...,0.182551,0.205616,0.306280,0.399641,0.606910,0.755213,0.886474,1.0,-0.594111,-0.412871
4,speech,calm,normal,Kids are talking by the door,1st,1,M,169769,0.137665,8.108948e-07,...,0.177069,0.177847,0.248765,0.428202,0.634815,0.759914,0.878014,1.0,0.126535,-0.620782


In [5]:
df_test.head()

,vocal_channel,emotion,emotional_intensity,statement,repetition,actor,sex,frame_count,sum,mean,...,stft_std_w4,stft_min_w4,stft_q01_w4,stft_q05_w4,stft_q25_w4,stft_q50_w4,stft_q75_w4,stft_q95_w4,stft_kur_w4,stft_skew_w4
0,speech,neutral,normal,Kids are talking by the door,1st,19,M,161762,0.475616,2.940224e-06,...,0.169929,0.222808,0.306697,0.466157,0.645931,0.789123,0.908316,1.0,-0.260279,-0.566556
1,speech,neutral,normal,Kids are talking by the door,2nd,19,M,155355,0.120148,7.733752e-07,...,0.216928,0.114567,0.170786,0.279347,0.557524,0.733053,0.866973,1.0,-0.428900,-0.587210
2,speech,neutral,normal,Dogs are sitting by the door,1st,19,M,160160,-0.599304,-3.741909e-06,...,0.227178,0.010295,0.023285,0.305395,0.577750,0.741576,0.889984,1.0,0.626916,-0.883317
3,speech,neutral,normal,Dogs are sitting by the door,2nd,19,M,158558,-0.080383,-5.069646e-07,...,0.179467,0.241405,0.368081,0.434591,0.620144,0.776419,0.904668,1.0,-0.825144,-0.383701
4,speech,calm,normal,Kids are talking by the door,1st,19,M,176176,0.152222,8.640319e-07,...,0.189124,0.217860,0.298851,0.393112,0.623309,0.772964,0.907035,1.0,-0.536669,-0.547354


## Standardization

## Train

In [6]:
df_train_num = df_train.select_dtypes(include="number").drop('actor', axis=1)
df_train_cat = df_train.select_dtypes(include=['object'])

scaler = MinMaxScaler()
df_train_num_scaled = pd.DataFrame(scaler.fit_transform(df_train_num), columns=df_train_num.columns)
df_train = pd.concat([df_train_num_scaled, df_train_cat], axis=1)

# Test

In [7]:
df_test_num = df_test.select_dtypes(include="number").drop('actor', axis=1)
df_test_cat = df_test.select_dtypes(include=['object'])

scaler = MinMaxScaler()
df_test_num_scaled = pd.DataFrame(scaler.fit_transform(df_test_num), columns=df_test_num.columns)
df_test = pd.concat([df_test_num_scaled, df_test_cat], axis=1)

In [8]:
def label_Encoder(array, label):
    new_array = np.zeros(len(array))
    for i in range(len(array)):
        if array[i] == label:
            new_array[i] = 1
    return new_array

## Vocal Channel

In [9]:
df_train_model_vc = df_train.copy()

In [10]:
df_test_model_vc = df_test.copy()

In [11]:
X_train_vc = df_train_model_vc.drop("vocal_channel", axis=1)
X_train_vc = np.array(pd.get_dummies(X_train_vc))
y_train_vc = label_Encoder(df_train_model_vc["vocal_channel"], 'speech')

In [12]:
X_train_vc.shape

(1828, 392)

In [13]:
X_test_vc = df_test_model_vc.drop("vocal_channel", axis=1)
X_test_vc = np.array(pd.get_dummies(X_test_vc))
y_test_vc = label_Encoder(df_test_model_vc["vocal_channel"], 'speech')

In [28]:
def bagging_classifier(X_train_array, y_train_array):
    
    
    param_grid = { "estimator__min_samples_split": loguniform(1e-2, 1e0),
                  "estimator__min_samples_leaf": uniform(0.001, 0.2),
                   "estimator__max_depth": list(range(2,200)),
                   "estimator__criterion": ["gini", "entropy", "log_loss"],
                  'max_samples':[0.5, 0.6, 0.7, 0.8], 
                  'max_features': np.arange(2,X_train_array.shape[1]),
    }


    clf=BaggingClassifier(DecisionTreeClassifier(), n_estimators = 100, random_state=42,)


    baggingclassifier=RandomizedSearchCV(clf,
                                    param_grid,
                                    cv=3,
                                    verbose=1,
                                    n_jobs=None,
                                    n_iter = 100,
                                    error_score='raise'
                      )


    baggingclassifier.fit(X_train_array,y_train_array)

    return baggingclassifier

## Bagging Classifier with Decision Tree

In [29]:
baggingclassifier_vocal_channel = bagging_classifier(X_train_vc, y_train_vc)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [30]:
print("Tuned Random Forest Parameters: {}".format(baggingclassifier_vocal_channel.best_params_))

Tuned Random Forest Parameters: {'estimator__criterion': 'log_loss', 'estimator__max_depth': 24, 'estimator__min_samples_leaf': 0.007504657654108948, 'estimator__min_samples_split': 0.018751878553982606, 'max_features': 275, 'max_samples': 0.7}


In [32]:
model_vocal_channel = BaggingClassifier(DecisionTreeClassifier (criterion = 'log_loss', 
                                        max_depth = 24, 
                                        min_samples_leaf = 0.007504657654108948, 
                                        min_samples_split = 0.018751878553982606), 
                                        max_features = 275, 
                                        max_samples = 0.7
                                        )
model_vocal_channel.fit(X_train_vc,y_train_vc)

BaggingClassifier(estimator=DecisionTreeClassifier(criterion='log_loss',
                                                   max_depth=24,
                                                   min_samples_leaf=0.007504657654108948,
                                                   min_samples_split=0.018751878553982606),
                  max_features=275, max_samples=0.7)

In [33]:
y_pred_dt_vc = model_vocal_channel.predict(X_test_vc)

In [34]:
print(classification_report(y_test_vc, y_pred_dt_vc))

              precision    recall  f1-score   support

         0.0       0.90      0.99      0.94       264
         1.0       0.99      0.92      0.96       360

    accuracy                           0.95       624
   macro avg       0.95      0.96      0.95       624
weighted avg       0.95      0.95      0.95       624



In [106]:
accuracy_bag_vc = accuracy_score(y_test_vc, y_pred_dt_vc)
f1_score_bag_vc = f1_score(y_test_vc, y_pred_dt_vc, average = 'weighted')

## Sex

In [35]:
df_train_model_s = df_train.copy()

In [36]:
df_test_model_s = df_test.copy()

In [37]:
X_train_s = df_train_model_s.drop("sex", axis=1)
X_train_s = np.array(pd.get_dummies(X_train_s))
y_train_s = label_Encoder(df_train_model_s["sex"], 'M')

In [38]:
X_test_s = df_test_model_s.drop("sex", axis=1)
X_test_s = np.array(pd.get_dummies(X_test_s))
y_test_s = label_Encoder(df_test_model_s["sex"], 'M')

## Bagging Classifier with Decision Tree

In [39]:
baggingclassifier_sex = bagging_classifier(X_train_s, y_train_s)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [40]:
print("Tuned Random Forest Parameters: {}".format(baggingclassifier_sex.best_params_))

Tuned Random Forest Parameters: {'estimator__criterion': 'log_loss', 'estimator__max_depth': 174, 'estimator__min_samples_leaf': 0.008885442726583957, 'estimator__min_samples_split': 0.02596052338716289, 'max_features': 107, 'max_samples': 0.6}


In [41]:
model_sex = BaggingClassifier(DecisionTreeClassifier (criterion = 'log_loss', 
                                        max_depth = 174, 
                                        min_samples_leaf = 0.008885442726583957, 
                                        min_samples_split = 0.02596052338716289), 
                                        max_features = 107, 
                                        max_samples = 0.6
                            )
model_sex.fit(X_train_s,y_train_s)

BaggingClassifier(estimator=DecisionTreeClassifier(criterion='log_loss',
                                                   max_depth=174,
                                                   min_samples_leaf=0.008885442726583957,
                                                   min_samples_split=0.02596052338716289),
                  max_features=107, max_samples=0.6)

In [42]:
y_pred_dt_s = model_sex.predict(X_test_s)

In [43]:
print(classification_report(y_test_s, y_pred_dt_s))

              precision    recall  f1-score   support

         0.0       0.98      0.73      0.84       312
         1.0       0.79      0.99      0.88       312

    accuracy                           0.86       624
   macro avg       0.89      0.86      0.86       624
weighted avg       0.89      0.86      0.86       624



In [109]:
accuracy_bag_s = accuracy_score(y_test_s, y_pred_dt_s)
f1_score_bag_s = f1_score(y_test_s, y_pred_dt_s, average = 'weighted')

## Emotional Intensity

In [44]:
df_train_model_ei = df_train.copy()

In [45]:
df_test_model_ei = df_test.copy()

In [46]:
X_train_ei = df_train_model_ei.drop("emotional_intensity", axis=1)
X_train_ei = np.array(pd.get_dummies(X_train_ei))
y_train_ei = label_Encoder(df_train_model_ei["emotional_intensity"], 'normal')

In [47]:
X_test_ei = df_test_model_ei.drop("emotional_intensity", axis=1)
X_test_ei = np.array(pd.get_dummies(X_test_ei))
y_test_ei = label_Encoder(df_test_model_ei["emotional_intensity"], 'normal')

## Bagging Classifier with Decision Tree

In [48]:
baggingclassifier_emotional_intensity = bagging_classifier(X_train_ei, y_train_ei)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [49]:
print("Tuned Random Forest Parameters: {}".format(baggingclassifier_emotional_intensity.best_params_))

Tuned Random Forest Parameters: {'estimator__criterion': 'log_loss', 'estimator__max_depth': 44, 'estimator__min_samples_leaf': 0.011315092471877251, 'estimator__min_samples_split': 0.013242987341748913, 'max_features': 313, 'max_samples': 0.8}


In [50]:
model_emotional_intensity = BaggingClassifier(DecisionTreeClassifier (criterion = 'log_loss', 
                                        max_depth = 44, 
                                        min_samples_leaf = 0.011315092471877251, 
                                        min_samples_split = 0.013242987341748913), 
                                        max_features = 313, 
                                        max_samples = 0.8
                            )
model_emotional_intensity.fit(X_train_ei,y_train_ei)

BaggingClassifier(estimator=DecisionTreeClassifier(criterion='log_loss',
                                                   max_depth=44,
                                                   min_samples_leaf=0.011315092471877251,
                                                   min_samples_split=0.013242987341748913),
                  max_features=313, max_samples=0.8)

In [51]:
y_pred_dt_ei = model_emotional_intensity.predict(X_test_ei)

In [52]:
print(classification_report(y_test_ei, y_pred_dt_ei))

              precision    recall  f1-score   support

         0.0       0.69      0.78      0.74       288
         1.0       0.79      0.71      0.75       336

    accuracy                           0.74       624
   macro avg       0.74      0.74      0.74       624
weighted avg       0.75      0.74      0.74       624



In [112]:
accuracy_bag_ei = accuracy_score(y_test_ei, y_pred_dt_ei)
f1_score_bag_ei = f1_score(y_test_ei, y_pred_dt_ei, average = 'weighted')

## Emotion

In [53]:
def label_encoder_e(array):
    new_array = np.zeros(len(array))
    for i in range(len(array)):
        if array[i] == "calm":
            new_array[i] = 1
        elif array[i] == "happy":
            new_array[i] = 2
        elif array[i] == "sad":
            new_array[i] = 3
        elif array[i] == "angry":
            new_array[i] = 4
        elif array[i] == "fearful":
            new_array[i] = 5
        elif array[i] == "disgust":
            new_array[i] = 6
        elif array[i] == "surprised":
            new_array[i] = 7
    return new_array

In [54]:
df_train_model_e = df_train.copy()

In [55]:
df_test_model_e = df_test.copy()

In [56]:
X_train_e = df_train_model_e.drop("emotion", axis=1)
X_train_e = np.array(pd.get_dummies(X_train_e))
y_train_e = label_encoder_e(df_train_model_e["emotion"])

In [57]:
X_test_e = df_test_model_e.drop("emotion", axis=1)
X_test_e = np.array(pd.get_dummies(X_test_e))
y_test_e = label_encoder_e(df_test_model_ei["emotion"])

## Bagging Classifier with Decision Tree

In [58]:
baggingclassifier_emotion = bagging_classifier(X_train_e, y_train_e)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [59]:
print("Tuned Random Forest Parameters: {}".format(baggingclassifier_emotion.best_params_))

Tuned Random Forest Parameters: {'estimator__criterion': 'log_loss', 'estimator__max_depth': 113, 'estimator__min_samples_leaf': 0.00578305249016755, 'estimator__min_samples_split': 0.038566137444645696, 'max_features': 102, 'max_samples': 0.8}


In [60]:
model_emotion = BaggingClassifier(DecisionTreeClassifier (criterion = 'log_loss', 
                                        max_depth = 113, 
                                        min_samples_leaf = 0.00578305249016755, 
                                        min_samples_split = 0.038566137444645696), 
                                        max_features = 102, 
                                        max_samples = 0.8 
                            )
model_emotion.fit(X_train_e,y_train_e)

BaggingClassifier(estimator=DecisionTreeClassifier(criterion='log_loss',
                                                   max_depth=113,
                                                   min_samples_leaf=0.00578305249016755,
                                                   min_samples_split=0.038566137444645696),
                  max_features=102, max_samples=0.8)

In [61]:
y_pred_dt_e = model_emotion.predict(X_test_e)

In [62]:
print(classification_report(y_test_e, y_pred_dt_e))

              precision    recall  f1-score   support

         0.0       0.40      0.17      0.24        48
         1.0       0.44      0.70      0.54        96
         2.0       0.33      0.38      0.35        96
         3.0       0.42      0.16      0.23        96
         4.0       0.43      0.84      0.57        96
         5.0       0.37      0.15      0.21        96
         6.0       0.46      0.33      0.39        48
         7.0       0.39      0.38      0.38        48

    accuracy                           0.41       624
   macro avg       0.40      0.39      0.36       624
weighted avg       0.40      0.41      0.37       624



In [115]:
accuracy_bag_e = accuracy_score(y_test_e, y_pred_dt_e)
f1_score_bag_e = f1_score(y_test_e, y_pred_dt_e, average = 'weighted')

## Some Emotion

In [64]:
df_train_model_es = df_train.copy()

In [65]:
df_test_model_es = df_test.copy()

In [66]:
df_train_subset = df_train_model_es[(df_train_model_es["emotion"] == 'happy')] 
df_train_subset1 = df_train_model_es[(df_train_model_es["emotion"] == 'sad')]
df_train_subset_e = pd.concat([df_train_subset, df_train_subset1])

In [67]:
label_encoder_es = preprocessing.LabelEncoder()
X_train_es = df_train_subset_e.drop("emotion", axis=1)
X_train_es = np.array(pd.get_dummies(X_train_es))
y_train_es = label_encoder_es.fit_transform(df_train_subset_e['emotion'])

In [68]:
df_test_subset = df_test_model_es[(df_test_model_es["emotion"] == 'happy')] 
df_test_subset1 = df_test_model_es[(df_test_model_es["emotion"] == 'sad')]
df_test_subset_e = pd.concat([df_test_subset, df_test_subset1])

In [69]:
X_test_es = df_test_subset_e.drop("emotion", axis=1)
X_test_es = np.array(pd.get_dummies(X_test_es))
y_test_es = label_encoder_es.fit_transform(df_test_subset_e["emotion"])

## Bagging Classifier with Decision Tree

In [70]:
baggingclassifier_emotion_subset = bagging_classifier(X_train_es, y_train_es)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [71]:
print("Tuned Random Forest Parameters: {}".format(baggingclassifier_emotion_subset.best_params_))

Tuned Random Forest Parameters: {'estimator__criterion': 'gini', 'estimator__max_depth': 55, 'estimator__min_samples_leaf': 0.0018912862017748324, 'estimator__min_samples_split': 0.010239334468147874, 'max_features': 361, 'max_samples': 0.7}


In [72]:
model_emotion_subset = BaggingClassifier(DecisionTreeClassifier (criterion = 'gini', 
                                        max_depth = 55, 
                                        min_samples_leaf = 0.0018912862017748324, 
                                        min_samples_split = 0.010239334468147874), 
                                        max_features = 361, 
                                        max_samples = 0.7 
                            )
model_emotion_subset.fit(X_train_es,y_train_es)

BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=55,
                                                   min_samples_leaf=0.0018912862017748324,
                                                   min_samples_split=0.010239334468147874),
                  max_features=361, max_samples=0.7)

In [73]:
y_pred_dt_es = model_emotion_subset.predict(X_test_es)

In [74]:
print(classification_report(y_test_es, y_pred_dt_es))

              precision    recall  f1-score   support

           0       0.63      0.59      0.61        96
           1       0.62      0.66      0.64        96

    accuracy                           0.62       192
   macro avg       0.63      0.62      0.62       192
weighted avg       0.63      0.62      0.62       192



## ADABOOST

In [140]:
def adaboost_classifier(X_train_array, y_train_array):
    
    
    param_grid = { "estimator__min_samples_split": loguniform(1e-2, 1e0),
                  "estimator__min_samples_leaf": uniform(0.001, 0.2),
                   "estimator__max_depth": list(range(2,200)),
                   "estimator__criterion": ["gini", "entropy", "log_loss"],
                  'learning_rate':[0.0001, 0.001, 0.01, 0.1, 1.0]
    }


    clf=AdaBoostClassifier(DecisionTreeClassifier(), n_estimators = 100, random_state=42)


    adaboostclassifier=RandomizedSearchCV(clf,
                                    param_grid,
                                    cv=3,
                                    verbose=1,
                                    n_jobs=None,
                                    #n_iter = 100,
                                    error_score='raise'
                      )


    adaboostclassifier.fit(X_train_array,y_train_array)

    return adaboostclassifier

## Vocal Channel

In [199]:
adaboostclassifier_vocal_channel = adaboost_classifier(X_train_vc, y_train_vc)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [200]:
print("Tuned Random Forest Parameters: {}".format(adaboostclassifier_vocal_channel.best_params_))

Tuned Random Forest Parameters: {'estimator__criterion': 'log_loss', 'estimator__max_depth': 77, 'estimator__min_samples_leaf': 0.0711766702686779, 'estimator__min_samples_split': 0.03250658875277545, 'learning_rate': 0.1}


In [202]:
model_adaboost_vc = AdaBoostClassifier(DecisionTreeClassifier (criterion = 'log_loss', 
                                       max_depth = 77, 
                                       min_samples_leaf = 0.0711766702686779, 
                                       min_samples_split = 0.03250658875277545), 
                                       learning_rate = 0.1
                                        )
model_adaboost_vc.fit(X_train_vc,y_train_vc)

AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='log_loss',
                                                    max_depth=77,
                                                    min_samples_leaf=0.0711766702686779,
                                                    min_samples_split=0.03250658875277545),
                   learning_rate=0.1)

In [203]:
y_adaboost_vc = model_adaboost_vc.predict(X_test_vc)

In [204]:
print(classification_report(y_test_vc, y_adaboost_vc))

              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96       264
         1.0       0.99      0.95      0.97       360

    accuracy                           0.96       624
   macro avg       0.96      0.97      0.96       624
weighted avg       0.97      0.96      0.96       624



In [205]:
accuracy_ab_vc = accuracy_score(y_test_vc, y_adaboost_vc)
f1_score_ab_vc = f1_score(y_test_vc, y_adaboost_vc, average = 'weighted')

## Sex

In [147]:
adaboostclassifier_sex = adaboost_classifier(X_train_s, y_train_s)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [148]:
print("Tuned Random Forest Parameters: {}".format(adaboostclassifier_sex.best_params_))

Tuned Random Forest Parameters: {'estimator__criterion': 'log_loss', 'estimator__max_depth': 197, 'estimator__min_samples_leaf': 0.09455790527444952, 'estimator__min_samples_split': 0.03232806672517797, 'learning_rate': 1.0}


In [149]:
model_adaboost_s = AdaBoostClassifier(DecisionTreeClassifier (criterion = 'log_loss', 
                                       max_depth = 197, 
                                       min_samples_leaf = 0.09455790527444952, 
                                       min_samples_split = 0.03232806672517797), 
                                       learning_rate = 1.0
                                        )
model_adaboost_s.fit(X_train_s,y_train_s)

AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='log_loss',
                                                    max_depth=197,
                                                    min_samples_leaf=0.09455790527444952,
                                                    min_samples_split=0.03232806672517797))

In [150]:
y_adaboost_s = model_adaboost_s.predict(X_test_s)

In [151]:
print(classification_report(y_test_s, y_adaboost_s))

              precision    recall  f1-score   support

         0.0       0.99      0.71      0.82       312
         1.0       0.77      0.99      0.87       312

    accuracy                           0.85       624
   macro avg       0.88      0.85      0.84       624
weighted avg       0.88      0.85      0.84       624



In [174]:
accuracy_ab_s = accuracy_score(y_test_s, y_adaboost_s)
f1_score_ab_s = f1_score(y_test_s, y_adaboost_s, average = 'weighted')

## Emotional Intensity

In [152]:
adaboostclassifier_emotional_intensity = adaboost_classifier(X_train_ei, y_train_ei)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [153]:
print("Tuned Random Forest Parameters: {}".format(adaboostclassifier_emotional_intensity.best_params_))

Tuned Random Forest Parameters: {'estimator__criterion': 'entropy', 'estimator__max_depth': 61, 'estimator__min_samples_leaf': 0.11945063353735935, 'estimator__min_samples_split': 0.017352444940485618, 'learning_rate': 0.1}


In [154]:
model_adaboost_ei = AdaBoostClassifier(DecisionTreeClassifier (criterion = 'entropy', 
                                       max_depth = 61, 
                                       min_samples_leaf = 0.11945063353735935, 
                                       min_samples_split = 0.017352444940485618), 
                                       learning_rate = 0.1
                                        )
model_adaboost_ei.fit(X_train_ei,y_train_ei)

AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='entropy',
                                                    max_depth=61,
                                                    min_samples_leaf=0.11945063353735935,
                                                    min_samples_split=0.017352444940485618),
                   learning_rate=0.1)

In [155]:
y_adaboost_ei = model_adaboost_ei.predict(X_test_ei)

In [156]:
print(classification_report(y_test_ei, y_adaboost_ei))

              precision    recall  f1-score   support

         0.0       0.75      0.73      0.74       288
         1.0       0.77      0.79      0.78       336

    accuracy                           0.76       624
   macro avg       0.76      0.76      0.76       624
weighted avg       0.76      0.76      0.76       624



In [177]:
accuracy_ab_ei = accuracy_score(y_test_ei, y_adaboost_ei)
f1_score_ab_ei = f1_score(y_test_ei, y_adaboost_ei, average = 'weighted')

## Emotion

In [157]:
adaboostclassifier_emotion = adaboost_classifier(X_train_e, y_train_e)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [158]:
print("Tuned Random Forest Parameters: {}".format(adaboostclassifier_emotion.best_params_))

Tuned Random Forest Parameters: {'estimator__criterion': 'gini', 'estimator__max_depth': 162, 'estimator__min_samples_leaf': 0.13196100651054382, 'estimator__min_samples_split': 0.1111401831158662, 'learning_rate': 0.01}


In [159]:
model_adaboost_e = AdaBoostClassifier(DecisionTreeClassifier (criterion = 'gini', 
                                       max_depth = 162, 
                                       min_samples_leaf = 0.13196100651054382, 
                                       min_samples_split = 0.1111401831158662), 
                                       learning_rate = 0.01
                                        )
model_adaboost_e.fit(X_train_e,y_train_e)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=162,
                                                    min_samples_leaf=0.13196100651054382,
                                                    min_samples_split=0.1111401831158662),
                   learning_rate=0.01)

In [160]:
y_adaboost_e = model_adaboost_e.predict(X_test_e)

In [161]:
print(classification_report(y_test_e, y_adaboost_e))

              precision    recall  f1-score   support

         0.0       0.44      0.08      0.14        48
         1.0       0.47      0.40      0.43        96
         2.0       0.32      0.60      0.42        96
         3.0       0.24      0.33      0.28        96
         4.0       0.46      0.72      0.56        96
         5.0       0.24      0.11      0.16        96
         6.0       0.67      0.08      0.15        48
         7.0       0.42      0.17      0.24        48

    accuracy                           0.36       624
   macro avg       0.41      0.31      0.30       624
weighted avg       0.39      0.36      0.32       624



In [180]:
accuracy_ab_e = accuracy_score(y_test_e, y_adaboost_e)
f1_score_ab_e = f1_score(y_test_e, y_adaboost_e, average = 'weighted')

## Some Emotion

In [162]:
adaboostclassifier_emotion_subset = adaboost_classifier(X_train_es, y_train_es)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [163]:
print("Tuned Random Forest Parameters: {}".format(adaboostclassifier_emotion_subset.best_params_))

Tuned Random Forest Parameters: {'estimator__criterion': 'gini', 'estimator__max_depth': 166, 'estimator__min_samples_leaf': 0.1983241640496444, 'estimator__min_samples_split': 0.07318168841760543, 'learning_rate': 1.0}


In [164]:
model_adaboost_es = AdaBoostClassifier(DecisionTreeClassifier (criterion = 'gini', 
                                       max_depth = 166, 
                                       min_samples_leaf = 0.1983241640496444, 
                                       min_samples_split = 0.07318168841760543), 
                                       learning_rate = 1.0
                                        )
model_adaboost_es.fit(X_train_es,y_train_es)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=166,
                                                    min_samples_leaf=0.1983241640496444,
                                                    min_samples_split=0.07318168841760543))

In [165]:
y_adaboost_es = model_adaboost_es.predict(X_test_es)

In [166]:
print(classification_report(y_test_es, y_adaboost_es))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86        96
           1       0.87      0.83      0.85        96

    accuracy                           0.85       192
   macro avg       0.85      0.85      0.85       192
weighted avg       0.85      0.85      0.85       192



## Metriche per Bagging Classifier

In [118]:
# vocal_channel

vocal_channel_metrics_b = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
vocal_channel_metrics_b.loc["Accuracy"] = [None, None, None, None]
vocal_channel_metrics_b.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
vocal_channel_metrics_b

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [121]:
vocal_channel_metrics_b.loc["Accuracy"][2] = accuracy_bag_vc
vocal_channel_metrics_b.loc["F1-Score (Weighted-Average)"][2] = f1_score_bag_vc

In [122]:
vocal_channel_metrics_b

,LG,RF,DTB,AB
Accuracy,None,None,0.950321,None
F1-Score (Weighted-Average),None,None,0.950561,None


In [123]:
# sex

sex_metrics_b = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
sex_metrics_b.loc["Accuracy"] = [None, None, None, None]
sex_metrics_b.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
sex_metrics_b

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [124]:
sex_metrics_b.loc["Accuracy"][2] = accuracy_bag_s
sex_metrics_b.loc["F1-Score (Weighted-Average)"][2] = f1_score_bag_s

In [125]:
sex_metrics_b

,LG,RF,DTB,AB
Accuracy,None,None,0.860577,None
F1-Score (Weighted-Average),None,None,0.858306,None


In [126]:
# emotional_intensity

emotional_intensity_metrics_b = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
emotional_intensity_metrics_b.loc["Accuracy"] = [None, None, None, None]
emotional_intensity_metrics_b.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
emotional_intensity_metrics_b

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [127]:
emotional_intensity_metrics_b.loc["Accuracy"][2] = accuracy_bag_ei
emotional_intensity_metrics_b.loc["F1-Score (Weighted-Average)"][2] = f1_score_bag_ei

In [128]:
emotional_intensity_metrics_b

,LG,RF,DTB,AB
Accuracy,None,None,0.740385,None
F1-Score (Weighted-Average),None,None,0.740673,None


In [129]:
# emotion

emotion_metrics_b = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
emotion_metrics_b.loc["Accuracy"] = [None, None, None, None]
emotion_metrics_b.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
emotion_metrics_b

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [130]:
emotion_metrics_b.loc["Accuracy"][2] = accuracy_bag_e
emotion_metrics_b.loc["F1-Score (Weighted-Average)"][2] = f1_score_bag_e

In [131]:
emotion_metrics_b

,LG,RF,DTB,AB
Accuracy,None,None,0.408654,None
F1-Score (Weighted-Average),None,None,0.369234,None


## Metriche per Adaboost

In [206]:
# vocal_channel

vocal_channel_metrics_ab = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
vocal_channel_metrics_ab.loc["Accuracy"] = [None, None, None, None]
vocal_channel_metrics_ab.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
vocal_channel_metrics_ab

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [207]:
vocal_channel_metrics_ab.loc["Accuracy"][3] = accuracy_ab_vc
vocal_channel_metrics_ab.loc["F1-Score (Weighted-Average)"][3] = f1_score_ab_vc

In [208]:
vocal_channel_metrics_ab

,LG,RF,DTB,AB
Accuracy,None,None,None,0.964744
F1-Score (Weighted-Average),None,None,None,0.964861


In [209]:
# sex

sex_metrics_ab = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
sex_metrics_ab.loc["Accuracy"] = [None, None, None, None]
sex_metrics_ab.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
sex_metrics_ab

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [210]:
sex_metrics_ab.loc["Accuracy"][3] = accuracy_ab_s
sex_metrics_ab.loc["F1-Score (Weighted-Average)"][3] = f1_score_ab_s

In [211]:
sex_metrics_ab

,LG,RF,DTB,AB
Accuracy,None,None,None,0.847756
F1-Score (Weighted-Average),None,None,None,0.844595


In [212]:
# emotional_intensity

emotional_intensity_metrics_ab = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
emotional_intensity_metrics_ab.loc["Accuracy"] = [None, None, None, None]
emotional_intensity_metrics_ab.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
emotional_intensity_metrics_ab

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [213]:
emotional_intensity_metrics_ab.loc["Accuracy"][3] = accuracy_ab_ei
emotional_intensity_metrics_ab.loc["F1-Score (Weighted-Average)"][3] = f1_score_ab_ei

In [214]:
emotional_intensity_metrics_ab

,LG,RF,DTB,AB
Accuracy,None,None,None,0.761218
F1-Score (Weighted-Average),None,None,None,0.760901


In [215]:
# emotion

emotion_metrics_ab = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
emotion_metrics_ab.loc["Accuracy"] = [None, None, None, None]
emotion_metrics_ab.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
emotion_metrics_ab

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [216]:
emotion_metrics_ab.loc["Accuracy"][3] = accuracy_ab_e
emotion_metrics_ab.loc["F1-Score (Weighted-Average)"][3] = f1_score_ab_e

In [217]:
emotion_metrics_ab

,LG,RF,DTB,AB
Accuracy,None,None,None,0.358974
F1-Score (Weighted-Average),None,None,None,0.324588
